In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
data = pd.read_csv('churn-bigml-20.csv')

In [4]:
data.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,True
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,True
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,False
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,False


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   667 non-null    object 
 1   Account length          667 non-null    int64  
 2   Area code               667 non-null    int64  
 3   International plan      667 non-null    object 
 4   Voice mail plan         667 non-null    object 
 5   Number vmail messages   667 non-null    int64  
 6   Total day minutes       667 non-null    float64
 7   Total day calls         667 non-null    int64  
 8   Total day charge        667 non-null    float64
 9   Total eve minutes       667 non-null    float64
 10  Total eve calls         667 non-null    int64  
 11  Total eve charge        667 non-null    float64
 12  Total night minutes     667 non-null    float64
 13  Total night calls       667 non-null    int64  
 14  Total night charge      667 non-null    fl

In [6]:
data.replace({False: 0, True: 1}, inplace=True)

In [7]:
data.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,0
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,1
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,1
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,0
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,0


# VIF

In [8]:
data_type = data
continuous_variables = ['Account length','Number vmail messages','Number vmail messages','Total day minutes','Total day calls','Total day charge','Total eve minutes','Total eve calls','Total eve charge','Total night minutes','Total night calls','Total night charge','Total intl minutes','Total intl calls','Total intl charge','Customer service calls']
categorical_variables = ['State']
        
target_variable = 'Churn'

In [9]:
def sklearn_vif(exogs, data):

        # initialize dictionaries
        vif_dict = {}

        # form input data for each exogenous variable
        for exog in exogs:
            #print(exog)
            not_exog = [i for i in exogs if i != exog]
            X, y = data[not_exog], data[exog]

            # extract r-squared from the fit
            r_squared = LinearRegression().fit(X, y).score(X, y)

            # calculate VIF
            vif = 1/(1 - r_squared)
            vif_dict[exog] = round(vif,2)

        # return VIF DataFrame
        df_vif = pd.DataFrame(vif_dict.items(),columns=['Variable_Name','VIF'])

        return df_vif

In [10]:
sklearn_vif(continuous_variables,data)

,Variable_Name,VIF
0,Account length,1.01
1,Number vmail messages,1.02
2,Total day minutes,10993269.68
3,Total day calls,1.02
4,Total day charge,10993306.82
5,Total eve minutes,2062802.82
6,Total eve calls,1.02
7,Total eve charge,2062810.56
8,Total night minutes,617268.92
9,Total night calls,1.02


# Event Rate Analysis

In [11]:
def event_rate_function(BSL_data,file_name="Event_Rate.csv"):
        continuous_variables = ['Account length','Number vmail messages','Number vmail messages','Total day minutes','Total day calls','Total day charge','Total eve minutes','Total eve calls','Total eve charge','Total night minutes','Total night calls','Total night charge','Total intl minutes','Total intl calls','Total intl charge','Customer service calls']
        categorical_variables = ['State']
        continuous_variables = list(set(continuous_variables) & set(BSL_data.columns.to_list()))
        categorical_variables = list(set(categorical_variables) & set(BSL_data.columns.to_list()))
        event_rates = {}

        k=0
        #for variable in numerical_variables:
        for variable in continuous_variables:
            #print(variable, k)
            split_df = np.array_split(BSL_data.sort_values(by=variable), 10)
            event_rate={}
            for i in range(10):
                try:
                    event_rate[i+1]=(split_df[i]['Churn'].value_counts()[1] / len(split_df[i]))
                except:
                    event_rate[i+1]=0            
            event_rates[variable] = list(event_rate.values()) 
            k=k+1

        #event_rates_dataframe = pd.DataFrame(event_rates)#, columns = ["Variable Name", "IV"])
        event_rates_dataframe = pd.DataFrame.from_dict(event_rates,orient='index')
        event_rates_dataframe = event_rates_dataframe.reset_index()
        event_rates_dataframe.columns = ["Variable_Name", '1','2','3','4','5','6','7','8','9','10']

        avg = (event_rates_dataframe.mean(axis=1).round(4)).tolist()
        std = (event_rates_dataframe.std(axis=1).round(4)).tolist()
        #print(event_rates_dataframe)
        event_rates_dataframe["Mean"] = avg
        event_rates_dataframe["Standard_Deviation"] = std

#         #print("Writing to dataframe")
#         event_rate_summary_path = f"{self.analysis_report_path}/{file_name}"
#         event_rates_dataframe.to_csv(event_rate_summary_path, index = False)

        return event_rates_dataframe

In [12]:
event_rate_function(data)

C:\Users\naveenk\AppData\Local\Temp/ipykernel_11400/22289766.py:27: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  avg = (event_rates_dataframe.mean(axis=1).round(4)).tolist()
C:\Users\naveenk\AppData\Local\Temp/ipykernel_11400/22289766.py:28: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  std = (event_rates_dataframe.std(axis=1).round(4)).tolist()


,Variable_Name,1,2,3,4,5,6,7,8,9,10,Mean,Standard_Deviation
0,Total day calls,0.149254,0.134328,0.149254,0.164179,0.089552,0.134328,0.089552,0.181818,0.181818,0.151515,0.1426,0.0324
1,Total intl charge,0.164179,0.059701,0.223881,0.194030,0.179104,0.149254,0.074627,0.075758,0.121212,0.181818,0.1424,0.0568
2,Total day charge,0.104478,0.134328,0.089552,0.134328,0.044776,0.044776,0.089552,0.060606,0.212121,0.515152,0.1430,0.1401
3,Number vmail messages,0.223881,0.164179,0.119403,0.164179,0.223881,0.208955,0.074627,0.075758,0.075758,0.090909,0.1422,0.0626
4,Total intl minutes,0.164179,0.059701,0.223881,0.194030,0.179104,0.149254,0.074627,0.075758,0.121212,0.181818,0.1424,0.0568
5,Total night minutes,0.089552,0.119403,0.104478,0.179104,0.164179,0.164179,0.149254,0.196970,0.106061,0.151515,0.1425,0.0357
6,Customer service calls,0.119403,0.074627,0.104478,0.089552,0.104478,0.104478,0.179104,0.136364,0.075758,0.439394,0.1428,0.1087
7,Total eve calls,0.104478,0.119403,0.149254,0.149254,0.134328,0.164179,0.164179,0.090909,0.121212,0.227273,0.1424,0.0386
8,Account length,0.119403,0.119403,0.164179,0.074627,0.238806,0.149254,0.134328,0.196970,0.136364,0.090909,0.1424,0.0485
9,Total night calls,0.119403,0.194030,0.164179,0.119403,0.134328,0.119403,0.179104,0.106061,0.166667,0.121212,0.1424,0.0307


# IV - Information Value

In [13]:
import pandas as pd

In [14]:
import numpy as np

In [15]:
def calc_iv(df, feature, target, pr=False):
    """
    Set pr=True to enable printing of output.
    
    Output: 
      * iv: float,
      * data: pandas.DataFrame
    """

    lst = []
    df[feature] = df[feature].fillna("NULL")

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature,                                                        # Variable
                    val,                                                            # Value
                    df[df[feature] == val].count()[feature],                        # All
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],  # Good (think: Fraud == 0)
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]]) # Bad (think: Fraud == 1)
    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Good', 'Bad'])

    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])

    data = data.sort_values(by=['Variable', 'Value'], ascending=[True, True])
    data.index = range(len(data.index))

    if pr:
        print(data)
        print('IV = ', data['IV'].sum())


    iv = data['IV'].sum()
    # print(iv)
    return iv, data

Looking at IV for some of the variables individually

In [16]:
df = pd.read_csv('churn-bigml-20.csv')
iv, data = calc_iv(df, 'Account length', 'Churn')

C:\Users\naveenk\Anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [17]:
iv

0.651588659510405

In [18]:
df = pd.read_csv('churn-bigml-20.csv')
iv, data = calc_iv(df, 'Total day minutes', 'Churn')

In [19]:
iv

0.27181434829901174

In [20]:
df = pd.read_csv('churn-bigml-20.csv')
iv, data = calc_iv(df, 'Total day charge', 'Churn')

In [21]:
iv

0.27181434829901174

In [22]:
df = pd.read_csv('churn-bigml-20.csv')
iv, data = calc_iv(df, 'Total day calls', 'Churn')

In [23]:
iv

0.45761199452446355

In [24]:
df = pd.read_csv('churn-bigml-20.csv')
iv, data = calc_iv(df, 'Total intl charge', 'Churn')

In [25]:
iv

0.5414837052010666